/dataset  
  /healthy       # Contains images of healthy leaves  
  /infected      # Contains images of diseased leaves  
  /pests         # Contains images of leaves with insects

In [2]:
import tensorflow as tf
from tensorflow import keras

# Load dataset from folders (make sure your dataset is structured correctly)
train_data = keras.preprocessing.image_dataset_from_directory(
    "dataset",
    image_size=(150, 150),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=123
)

val_data = keras.preprocessing.image_dataset_from_directory(
    "dataset",
    image_size=(150, 150),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Create a simple AI model
model = keras.Sequential([
    keras.layers.Rescaling(1./255, input_shape=(150, 150, 3)),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(3, activation="softmax")  # 3 categories: healthy, infected, pests
])

# Compile and train the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_data, validation_data=val_data, epochs=5)

# Save the trained model
model.save("pest_detection_model.h5")

# Convert model to TensorFlow Lite for Raspberry Pi
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("pest_detection_model.tflite", "wb") as f:
    f.write(tflite_model)

2025-03-16 01:46:38.449893: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 01:46:38.800370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742069798.902081    5030 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742069798.915377    5030 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742069798.961545    5030 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

NotFoundError: Could not find directory dataset

In [ ]:
import cv2
import numpy as np
import tensorflow.lite as tflite

# Load AI Model
interpreter = tflite.Interpreter(model_path="pest_detection_model.tflite")
interpreter.allocate_tensors()

# Load Raspberry Pi Camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Resize image for AI model
    img_resized = cv2.resize(frame, (150, 150))
    img_resized = np.expand_dims(img_resized, axis=0).astype(np.float32)

    # Run AI model
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], img_resized)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Print Results
    if prediction[0][0] > 0.5:
        print("Pest detected!")
    else:
        print("No pests detected.")

    cv2.imshow("Live Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()